In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

threshold = 1e-4
train = pd.read_csv('/kaggle/input/gemma-rewrite-nbroad/nbroad-v1.csv')
test = pd.read_csv('/kaggle/input/llm-prompt-recovery/test.csv')
# test = train.iloc[:5,:]
sample_sub = pd.read_csv('/kaggle/input/llm-prompt-recovery/sample_submission.csv')

vectorizer = TfidfVectorizer(ngram_range=(1, 1), sublinear_tf=True, max_features=40000)
n_test,n_train = test.shape[0],train.shape[0]
x_test = test.original_text.tolist() + test.rewritten_text.tolist()
x_train = train.original_text.tolist() + train.rewritten_text.tolist()

# x_test = vectorizer.fit_transform(x_test).toarray()
vectorizer.fit(x_test)
x_test = vectorizer.transform(x_test).toarray()
x_train = vectorizer.transform(x_train).toarray()
x_test = x_test[:n_test] - x_test[n_test:]
x_train = x_train[:n_train] - x_train[n_train:]

train_idx = x_train.mean(0) >= threshold
test_idx = x_test.mean(0) >= threshold
idx = np.logical_and(train_idx,test_idx)
# dist = np.abs(x_train - x_test[:,None]).sum(-1) # L1 norm of shape (n_test,n_train)
dist = np.abs(x_train[:,idx] - x_test[:,idx][:,None]).sum(-1) # L1 norm of shape (n_test,n_train)

temp = []
for i in range(n_test):
    train['dist'] = dist[i]
    out = train.groupby(['rewrite_prompt'])['dist'].mean()
    pred = out.index[out.argmin()]
    temp.append(pred)

sample_sub['rewrite_prompt'] = temp
sample_sub.to_csv('submission.csv',header=True,index=False)

In [2]:
# import pandas as pd
# import numpy as np
# from sklearn.feature_extraction.text import TfidfVectorizer
# try:
#     threshold = 0
#     train = pd.read_csv('/kaggle/input/gemma-rewrite-nbroad/nbroad-v1.csv')
#     test = pd.read_csv('/kaggle/input/llm-prompt-recovery/test.csv')
#     # test = train.iloc[:5,:]
#     sample_sub = pd.read_csv('/kaggle/input/llm-prompt-recovery/sample_submission.csv')

#     vectorizer = TfidfVectorizer(ngram_range=(1, 3), sublinear_tf=True)
#     n_test,n_train = test.shape[0],train.shape[0]
#     x_test = test.original_text.tolist() + test.rewritten_text.tolist()
#     x_train = train.original_text.tolist() + train.rewritten_text.tolist()
#     assert len(x_test) == (n_test*2)
# #     x_test = vectorizer.fit_transform(x_test).toarray()
# #     x_train = vectorizer.transform(x_train).toarray()
# #     x_test = x_test[:n_test] - x_test[n_test:]
# #     x_train = x_train[:n_train] - x_train[n_train:]
#     sample_sub['rewrite_prompt'] = ["" for i in range(n_test)]
#     sample_sub.to_csv('submission.csv',header=True,index=False)
# #     train_idx = x_train.mean(0) >= threshold
# #     test_idx = x_test.mean(0) >= threshold
# #     idx = np.logical_and(train_idx,test_idx)
# #     dist = np.abs(x_train[:,idx] - x_test[:,idx][:,None]).sum(-1) # L1 norm of shape (n_test,n_train)

# except Exception as e:
# #     temp = []
# #     for i in range(n_test):
# #         train['dist'] = dist[i]
# #         out = train.groupby(['rewrite_prompt'])['dist'].mean()
# #         pred = out.index[out.argmin()]
# #         temp.append(pred)
# #     sample_sub['rewrite_prompt'] = temp
#     sample_sub.to_csv('submission.csv',header=True,index=False)